In [1]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

# For importing .xml files
import xml.etree.ElementTree as ET

# For handeling dataframes
import pandas as pd
import numpy as np
import re                                  # For regular expression operations
import string                              # For string operations
import math
import gc

# For text preprocessing
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # For stop words that come with NLTK

# from nltk.stem import PorterStemmer as ps  # For stemming

# For building neural netwrok models
import sklearn
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam

# For model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import time
import random

# For text agumenters
import os
os.environ["MODEL_DIR"] = '../model'
import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import nlpaug.model.word_stats as nmw

from nlpaug.util import Action
import tqdm
from tqdm import tqdm # To show the progress bar
from joblib import Parallel, delayed

nltk.download('averaged_perceptron_tagger')  # 
nltk.download('wordnet')                     # For Synonym augmenter
nltk.download('punkt')                       # For BackTranslation augmenter

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
# pip install torch torchvision

In [3]:
# pip install nlpaug

In [4]:
# import sys

In [5]:
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [6]:
# Make the outcome variables (disease types) binary
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [7]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalences = (binary_labels.aggregate('sum')/1237).iloc[1:,]

# Combine the text data and the labels
df = text_all.merge(binary_labels, on="id", how="left")

In [8]:
########################################################################
# NLP Preprocessing
########################################################################
# Download the stopwords from NLTK
nltk.download('stopwords')
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Clean text
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in stopwords_english) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    return(text)

In [10]:
#######################################################################
# Functions
#######################################################################

# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
#     pred = model.predict_classes(X)
    pred = (model.predict(X) > 0.5).astype("int32")[:,1]
    acc = np.sum(y == pred)/len(pred)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred[:,1]),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred[:,1])
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [11]:
def _tokenizer(text, token_pattern=r"(?u)\b\w\w+\b"):
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(text)

In [12]:
# Arguments for word augmenter
aug_min_2=100
aug_max_2=1000
top_k=500 
aug_p=0.3

# Define the augmenters
# Character Augmenters
augs = []

# TFIDF Augmenter
augs.append(naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action='insert',
                              aug_min=aug_min_2,aug_max=aug_max_2, aug_p=aug_p, top_k=top_k))
# TFIDF Augmenter
augs.append(naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action='substitute',
                              aug_min=aug_min_2,aug_max=aug_max_2, aug_p=aug_p, top_k=top_k))

In [13]:
# Create the Transformer model
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_length, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = layers.Embedding(input_dim=max_length, output_dim=embedding_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.3):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embedding_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    

def transformer_model(vocab_size, embedding_dim,max_length,dropout_rate):   
    inputs = layers.Input(shape=(max_length,))
    embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embedding_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embedding_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(2, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    
    return(model)

In [18]:
#######################################################################
# Modeling
#######################################################################

# Set the number of epochs and the batch size
epochs = 20 ###################################################################
batch_size = 32
# tokenizer = Tokenizer(num_words=5000) # get the frequency of all tokens and use the 5000 most common ones
max_length = 557 # assumes the first 522 words are the most important, and make all sequences with length 525
dropout_rate = 0.3
embedding_dim = 200
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

time_1 = time.time()

X = df['text']
pct = 0.4

with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
    fd.write(f'Other-strategies_Prevalence between 0.1-0.3 and augment to {pct}_TFIDF\n')

for i in range(2,10):
    # Dummify the label variable event
    event_categorical = to_categorical(df['This_Disease_'+str(i)])
    prevalence = prevalences.iloc[i]
    dis_type = disease_types[i]
    j = 0 # to keep track of the iteration number
    time_start = time.time()
    
    # Run the model 10 times with a different split each time  
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=0)
    for train_index, test_index in sss.split(X, event_categorical):
        time_s = time.time()
        
        j += 1
        iteration = "iter" + str(j)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
          
        
        # Select the samples to be augmented 
        if prevalence < 0.5: # If disease presence is the minority
            x_aug_0 = x_train[y_train[:,1]==1] # Augment disease presence cases
            y_aug_0 = y_train[y_train[:,1]==1,0]

        else: # If disease absence is the minority
            x_aug_0 = x_train[y_train[:,0]==1] # Augment disease absence cases
            y_aug_0 = y_train[y_train[:,0]==1,0]
            
        
        x_aug_0  = [re.sub("(\d+|\n)", " ", elem).strip() for elem in x_aug_0]
        x_aug_0 = [str(x).lower() for x in x_aug_0]

        # Tokenize input
        train_x_tokens = [_tokenizer(x) for x in x_aug_0]

        # Train TF-IDF model
        tfidf_model = nmw.TfIdf()
        tfidf_model.train(train_x_tokens)
        tfidf_model.save('.')    

        x_aug_0 = np.array(x_aug_0)

        # Randomly augment a predetermined number of the selected cases
        length = len(x_aug_0) # Total number of texts availble for aumentation
#         num = math.ceil(abs(prevalence - 0.5)*len(x_train))*2 # Number of samples needed to balance the classes in the training set
        num = math.ceil(abs(pct*len(x_train) - sum(y_train[:,1]))/(1-pct))
        
        if num <= length:
            # If the number of texts to be augmented is larger than the number of samples needed
            samples = np.random.choice(range(length), size=num, replace=False) # Generate random indices without replacement
        else:
            samples = np.random.choice(range(length), size=num, replace=True) # Generate random indices with replacement
               

        for k in range(2): # Go through each TFIDF methods
            DA_start = time.time()
            aug = augs[k]
            
            new_text = []
            new_y_0 = []
            new_y_1 = []
            
            for ii in samples:   
                text = x_aug_0[ii]
                label_aug = y_aug_0[ii]
                augmented_text = aug.augment(text)
                    
                new_text.append(augmented_text)
                new_y_0.append(label_aug) # Assign the label of the original text to the augmented text
                new_y_1.append(1-label_aug) # Assign the label of the original text to the augmented text

            # Append the augmented texts and their corresponding labels to the original data
            x_aug = np.append(x_train, new_text)
            y_0 = np.append(y_train[:,0], new_y_0)
            y_1 = np.append(y_train[:,1], new_y_1)
            y_aug = (np.vstack((y_0,y_1))).T
            #################################################################################

            # Shuffle the order of the augmented dataset
            augmented = pd.DataFrame((np.vstack((x_aug, y_aug[:,0], y_aug[:,1]))).T)
            augmented = augmented.sample(frac=1).reset_index(drop=True)

            DA_time = time.time() - DA_start
            augmenter = "aug_TFIDF_"+str(k)
            with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{augmenter},DA Running Time:,{DA_time}\n')

            # Convert the dataframe to numpy arrays
            x_train_1 = np.array(augmented.iloc[:,0])
            y_train_1 = np.array(augmented.iloc[:,[1,2]])
            y_train_1 = y_train_1.astype('float32')

            # Run CNN model
            cnn_time_start = time.time()

            # Preprocess the text
            vfunc = np.vectorize(preprocessing)
            x_train_1 = vfunc(x_train_1)
            x_test_1 = vfunc(x_test)

            # Tokenize the text   
            tokenizer = Tokenizer(num_words=5000) # get the frequency of all tokens and use the 5000 most common ones
            tokenizer.fit_on_texts(x_train_1)
            x_train_1 = tokenizer.texts_to_sequences(x_train_1)
            x_test_1 = tokenizer.texts_to_sequences(x_test_1)
            vocab_size = len(tokenizer.word_index) + 1 # plus the reserved index 0
            word_index = tokenizer.word_index

            # Pad the sequences with 0's
            x_train_1 = pad_sequences(x_train_1, padding='post', maxlen=max_length) 
            x_test_1 = pad_sequences(x_test_1, padding='post', maxlen=max_length)

            # Fit the Transformer model
            mymodel = transformer_model(vocab_size, embedding_dim,max_length,dropout_rate)
            mymodel.fit(x_train_1, y_train_1, epochs=epochs, batch_size=batch_size)

            # Collect and log evaluation metrics
            auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(mymodel, x_test_1, y_test[:,1])
            cnn_time = time.time() - cnn_time_start

            with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{augmenter},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{cnn_time},{len(x_aug)}\n')

            del tokenizer, mymodel, vocab_size, word_index, x_aug, y_aug, augmented, x_train_1, x_test_1, y_train_1
            gc.collect()
            
    running_time = time.time() - time_start
    with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
            fd.write(f'{dis_type},10 iteration running time,{running_time}\n')

Epoch 1/20
42/42 [==============================] - 57s 1s/step - loss: 0.6108 - auc: 0.7408
Epoch 2/20
42/42 [==============================] - 56s 1s/step - loss: 0.1537 - auc: 0.9844
Epoch 3/20
42/42 [==============================] - 56s 1s/step - loss: 0.0302 - auc: 0.9996
Epoch 4/20
42/42 [==============================] - 56s 1s/step - loss: 0.0071 - auc: 1.0000
Epoch 5/20
42/42 [==============================] - 56s 1s/step - loss: 0.0026 - auc: 1.0000
Epoch 6/20
42/42 [==============================] - 56s 1s/step - loss: 6.2573e-04 - auc: 1.0000
Epoch 7/20
42/42 [==============================] - 56s 1s/step - loss: 2.2365e-04 - auc: 1.0000
Epoch 8/20
42/42 [==============================] - 56s 1s/step - loss: 1.5312e-04 - auc: 1.0000
Epoch 9/20
42/42 [==============================] - 56s 1s/step - loss: 1.3082e-04 - auc: 1.0000
Epoch 10/20
42/42 [==============================] - 56s 1s/step - loss: 1.1059e-04 - auc: 1.0000
Epoch 11/20
42/42 [==============================

42/42 [==============================] - 51s 1s/step - loss: 0.0057 - auc: 1.0000
Epoch 7/20
42/42 [==============================] - 51s 1s/step - loss: 0.0085 - auc: 1.0000
Epoch 8/20
42/42 [==============================] - 51s 1s/step - loss: 0.0108 - auc: 0.9999
Epoch 9/20
42/42 [==============================] - 51s 1s/step - loss: 0.0276 - auc: 0.9988
Epoch 10/20
42/42 [==============================] - 51s 1s/step - loss: 0.0015 - auc: 1.0000
Epoch 11/20
42/42 [==============================] - 51s 1s/step - loss: 3.3444e-04 - auc: 1.0000
Epoch 12/20
42/42 [==============================] - 51s 1s/step - loss: 1.4975e-04 - auc: 1.0000
Epoch 13/20
42/42 [==============================] - 51s 1s/step - loss: 1.3291e-04 - auc: 1.0000
Epoch 14/20
42/42 [==============================] - 51s 1s/step - loss: 1.2235e-04 - auc: 1.0000
Epoch 15/20
42/42 [==============================] - 51s 1s/step - loss: 1.0684e-04 - auc: 1.0000
Epoch 16/20
42/42 [==============================] - 51

42/42 [==============================] - 54s 1s/step - loss: 1.6787e-04 - auc: 1.0000
Epoch 12/20
42/42 [==============================] - 54s 1s/step - loss: 1.4401e-04 - auc: 1.0000
Epoch 13/20
42/42 [==============================] - 54s 1s/step - loss: 1.1563e-04 - auc: 1.0000
Epoch 14/20
42/42 [==============================] - 54s 1s/step - loss: 1.1046e-04 - auc: 1.0000
Epoch 15/20
42/42 [==============================] - 54s 1s/step - loss: 9.7641e-05 - auc: 1.0000
Epoch 16/20
42/42 [==============================] - 54s 1s/step - loss: 1.0183e-04 - auc: 1.0000
Epoch 17/20
42/42 [==============================] - 54s 1s/step - loss: 7.8956e-05 - auc: 1.0000
Epoch 18/20
42/42 [==============================] - 54s 1s/step - loss: 8.2214e-05 - auc: 1.0000
Epoch 19/20
42/42 [==============================] - 54s 1s/step - loss: 7.2137e-05 - auc: 1.0000
Epoch 20/20
42/42 [==============================] - 54s 1s/step - loss: 5.9411e-05 - auc: 1.0000
Epoch 1/20
42/42 [==============

42/42 [==============================] - 53s 1s/step - loss: 1.3625e-04 - auc: 1.0000
Epoch 17/20
42/42 [==============================] - 53s 1s/step - loss: 1.2002e-04 - auc: 1.0000
Epoch 18/20
42/42 [==============================] - 53s 1s/step - loss: 1.1804e-04 - auc: 1.0000
Epoch 19/20
42/42 [==============================] - 53s 1s/step - loss: 9.9951e-05 - auc: 1.0000
Epoch 20/20
42/42 [==============================] - 53s 1s/step - loss: 9.2068e-05 - auc: 1.0000
Epoch 1/20
42/42 [==============================] - 65s 2s/step - loss: 0.6005 - auc: 0.7381
Epoch 2/20
42/42 [==============================] - 64s 2s/step - loss: 0.2165 - auc: 0.9707
Epoch 3/20
42/42 [==============================] - 64s 2s/step - loss: 0.0509 - auc: 0.9987
Epoch 4/20
42/42 [==============================] - 63s 2s/step - loss: 0.0086 - auc: 1.0000
Epoch 5/20
42/42 [==============================] - 63s 1s/step - loss: 0.0019 - auc: 1.0000
Epoch 6/20
42/42 [==============================] - 63s 2

42/42 [==============================] - 61s 1s/step - loss: 0.5530 - auc: 0.7872
Epoch 2/20
42/42 [==============================] - 60s 1s/step - loss: 0.1567 - auc: 0.9841
Epoch 3/20
42/42 [==============================] - 60s 1s/step - loss: 0.0480 - auc: 0.9985
Epoch 4/20
42/42 [==============================] - 60s 1s/step - loss: 0.0059 - auc: 1.0000
Epoch 5/20
42/42 [==============================] - 60s 1s/step - loss: 9.4853e-04 - auc: 1.0000
Epoch 6/20
42/42 [==============================] - 60s 1s/step - loss: 5.0923e-04 - auc: 1.0000
Epoch 7/20
42/42 [==============================] - 60s 1s/step - loss: 3.4583e-04 - auc: 1.0000
Epoch 8/20
42/42 [==============================] - 60s 1s/step - loss: 2.6887e-04 - auc: 1.0000
Epoch 9/20
42/42 [==============================] - 60s 1s/step - loss: 2.0635e-04 - auc: 1.0000
Epoch 10/20
42/42 [==============================] - 60s 1s/step - loss: 1.7712e-04 - auc: 1.0000
Epoch 11/20
42/42 [==============================] - 60s

...

39/39 [==============================] - 57s 1s/step - loss: 0.0545 - auc: 0.9982
Epoch 5/20
39/39 [==============================] - 56s 1s/step - loss: 0.0274 - auc: 0.9991
Epoch 6/20
39/39 [==============================] - 57s 1s/step - loss: 0.0075 - auc: 1.0000
Epoch 7/20
39/39 [==============================] - 56s 1s/step - loss: 0.0040 - auc: 1.0000
Epoch 8/20
39/39 [==============================] - 56s 1s/step - loss: 0.0013 - auc: 1.0000
Epoch 9/20
39/39 [==============================] - 56s 1s/step - loss: 3.2858e-04 - auc: 1.0000
Epoch 10/20
39/39 [==============================] - 56s 1s/step - loss: 1.8241e-04 - auc: 1.0000
Epoch 11/20
39/39 [==============================] - 57s 1s/step - loss: 1.4641e-04 - auc: 1.0000
Epoch 12/20
39/39 [==============================] - 57s 1s/step - loss: 1.4057e-04 - auc: 1.0000
Epoch 13/20
39/39 [==============================] - 57s 1s/step - loss: 1.2317e-04 - auc: 1.0000
Epoch 14/20
39/39 [==============================] - 56s 

39/39 [==============================] - 57s 1s/step - loss: 0.0010 - auc: 1.0000
Epoch 10/20
39/39 [==============================] - 57s 1s/step - loss: 4.1026e-04 - auc: 1.0000
Epoch 11/20
39/39 [==============================] - 58s 1s/step - loss: 2.8087e-04 - auc: 1.0000
Epoch 12/20
39/39 [==============================] - 58s 1s/step - loss: 2.2288e-04 - auc: 1.0000
Epoch 13/20
39/39 [==============================] - 58s 1s/step - loss: 2.0918e-04 - auc: 1.0000
Epoch 14/20
39/39 [==============================] - 58s 1s/step - loss: 1.6426e-04 - auc: 1.0000
Epoch 15/20
39/39 [==============================] - 57s 1s/step - loss: 1.5932e-04 - auc: 1.0000
Epoch 16/20
39/39 [==============================] - 57s 1s/step - loss: 1.4300e-04 - auc: 1.0000
Epoch 17/20
39/39 [==============================] - 57s 1s/step - loss: 1.3031e-04 - auc: 1.0000
Epoch 18/20
39/39 [==============================] - 57s 1s/step - loss: 1.2093e-04 - auc: 1.0000
Epoch 19/20
39/39 [=================

39/39 [==============================] - 55s 1s/step - loss: 1.7042e-04 - auc: 1.0000
Epoch 15/20
39/39 [==============================] - 55s 1s/step - loss: 1.6286e-04 - auc: 1.0000
Epoch 16/20
39/39 [==============================] - 55s 1s/step - loss: 1.3134e-04 - auc: 1.0000
Epoch 17/20
39/39 [==============================] - 55s 1s/step - loss: 1.1455e-04 - auc: 1.0000
Epoch 18/20
39/39 [==============================] - 55s 1s/step - loss: 1.1907e-04 - auc: 1.0000
Epoch 19/20
39/39 [==============================] - 55s 1s/step - loss: 9.2822e-05 - auc: 1.0000
Epoch 20/20
39/39 [==============================] - 55s 1s/step - loss: 8.7476e-05 - auc: 1.0000
Epoch 1/20
39/39 [==============================] - 66s 2s/step - loss: 0.6541 - auc: 0.6677
Epoch 2/20
39/39 [==============================] - 65s 2s/step - loss: 0.4051 - auc: 0.9001
Epoch 3/20
39/39 [==============================] - 65s 2s/step - loss: 0.1646 - auc: 0.9852
Epoch 4/20
39/39 [=============================

39/39 [==============================] - 64s 2s/step - loss: 1.1380e-04 - auc: 1.0000
Epoch 20/20
39/39 [==============================] - 63s 2s/step - loss: 1.1146e-04 - auc: 1.0000
Epoch 1/20
39/39 [==============================] - 71s 2s/step - loss: 0.6355 - auc: 0.7007
Epoch 2/20
39/39 [==============================] - 70s 2s/step - loss: 0.3428 - auc: 0.9286
Epoch 3/20
39/39 [==============================] - 70s 2s/step - loss: 0.1422 - auc: 0.9875
Epoch 4/20
39/39 [==============================] - 70s 2s/step - loss: 0.0467 - auc: 0.9989
Epoch 5/20
39/39 [==============================] - 70s 2s/step - loss: 0.0360 - auc: 0.9991
Epoch 6/20
39/39 [==============================] - 70s 2s/step - loss: 0.0161 - auc: 0.9999
Epoch 7/20
39/39 [==============================] - 70s 2s/step - loss: 0.0020 - auc: 1.0000
Epoch 8/20
39/39 [==============================] - 70s 2s/step - loss: 5.7807e-04 - auc: 1.0000
Epoch 9/20
39/39 [==============================] - 70s 2s/step - lo

39/39 [==============================] - 74s 2s/step - loss: 0.0608 - auc: 0.9975
Epoch 5/20
39/39 [==============================] - 75s 2s/step - loss: 0.0179 - auc: 0.9994
Epoch 6/20
39/39 [==============================] - 75s 2s/step - loss: 0.0187 - auc: 0.9998
Epoch 7/20
39/39 [==============================] - 74s 2s/step - loss: 0.0078 - auc: 1.0000
Epoch 8/20
39/39 [==============================] - 75s 2s/step - loss: 0.0085 - auc: 1.0000
Epoch 9/20
39/39 [==============================] - 75s 2s/step - loss: 7.9842e-04 - auc: 1.0000
Epoch 10/20
39/39 [==============================] - 74s 2s/step - loss: 2.9225e-04 - auc: 1.0000
Epoch 11/20
39/39 [==============================] - 75s 2s/step - loss: 2.2601e-04 - auc: 1.0000
Epoch 12/20
39/39 [==============================] - 74s 2s/step - loss: 1.9552e-04 - auc: 1.0000
Epoch 13/20
39/39 [==============================] - 74s 2s/step - loss: 1.7613e-04 - auc: 1.0000
Epoch 14/20
39/39 [==============================] - 75s 

### Augment to 50% and then raise both

In [28]:
#######################################################################
# Modeling
#######################################################################

# Set the number of epochs and the batch size
epochs = 20 ###################################################################
# epochs = 2
batch_size = 32
max_length = 557
dropout_rate = 0.3
embedding_dim = 200
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

time_1 = time.time()

X = df['text']

pct = 0.1

with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
    fd.write(f'Prevalence > 0.6 and augment to 50% and raise {pct} each class \n')
       
for i in range(14,16):

    event_categorical = to_categorical(df['This_Disease_'+str(i)])
    prevalence = prevalences.iloc[i]
    dis_type = disease_types[i]
    j = 0 # to keep track of the iteration number
    time_start = time.time()
    
    # Run the model 10 times with a different split each time  

    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=0)##################################
    for train_index, test_index in sss.split(X, event_categorical):
        time_s = time.time()
        
        j += 1
        iteration = "iter" + str(j)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
        # Select the samples from minority class to be augmented
        if prevalence < 0.5: # If disease presence is the minority
            x_aug_0_minority = x_train[y_train[:,1]==1] # Augment disease presence cases
            y_aug_0_minority = y_train[y_train[:,1]==1,0]
            
            x_aug_0_majority = x_train[y_train[:,1]==0] # Augment disease absent cases
            y_aug_0_majority = y_train[y_train[:,1]==0,0]
            
            x_aug_0_tfidf  = [re.sub("(\d+|\n)", " ", elem).strip() for elem in x_aug_0_majority]
            x_aug_0_tfidf = [str(x).lower() for x in x_aug_0_tfidf]

            # Tokenize input
            train_x_tokens = [_tokenizer(x) for x in x_aug_0_tfidf]

        else: # If disease absence is the minority
            x_aug_0_minority = x_train[y_train[:,0]==1] # Augment disease absence cases
            y_aug_0_minority = y_train[y_train[:,0]==1,0]
            
            x_aug_0_majority = x_train[y_train[:,0]==0] # Augment disease presence cases
            y_aug_0_majority = y_train[y_train[:,0]==0,0]
            
            length = len(x_aug_0_minority) # Total number of texts availble for aumentation

            x_aug_0_tfidf  = [re.sub("(\d+|\n)", " ", elem).strip() for elem in x_aug_0_majority]
            x_aug_0_tfidf = [str(x).lower() for x in x_aug_0_tfidf]

            # Tokenize input
            train_x_tokens = [_tokenizer(x) for x in x_aug_0_tfidf]
        
        ###########################################
                # Train TF-IDF model
        tfidf_model = nmw.TfIdf()
        tfidf_model.train(train_x_tokens)
        tfidf_model.save('.')   
        ###########################################
        
        x_aug_0_minority = np.array(x_aug_0_minority)
        x_aug_0_majority = np.array(x_aug_0_majority)

        # Randomly augment a predetermined number of the selected cases
        
        num = math.ceil(abs(prevalence - 0.5)*len(x_train))*2 # Number of samples needed to balance the classes in the training set
        # Add 10% after balanced
        num_raise = math.ceil(pct *(num + len(x_train))/2) 
        
        num_minority = num + num_raise
        num_majority = num_raise
        # num = math.ceil(abs(0.4*len(x_train) - sum(y_train[:,1]))/0.6) # Augment to 20%
        
        length = len(x_aug_0_minority) # Total number of texts availble for aumentation
        if num_minority <= length:
            # If the number of texts to be augmented is larger than the number of samples needed
            samples_minority = np.random.choice(range(length), size=num_minority, replace=False) # Generate random indices without replacement
        else:
            samples_minority = np.random.choice(range(length), size=num_minority, replace=True) # Generate random indices with replacement
 
        if num_majority <= length:
            # If the number of texts to be augmented is larger than the number of samples needed
            samples_majority = np.random.choice(range(length), size=num_majority, replace=False) # Generate random indices without replacement
        else:
            samples_majority = np.random.choice(range(length), size=num_majority, replace=True)

        # Select the samples from majority class to be augmented
        
        
        # Augment the training set             
        # Go through all 17 augmenters: (run Synonym, TFIDF, and Backtranslation separately)
#         for k in range(3):
        for k in range(len(augs)):
            DA_start = time.time()
            aug = augs[k] # Assign the kth augmenter

            new_text_minority = []
            new_y_0_minority = []
            new_y_1_minority = []
                       

            for ii in samples_minority: # Go through each sample to be augmented   
                text_minority = x_aug_0_minority[ii]
                label_aug_minority = y_aug_0_minority[ii]
                augmented_text_minority = aug.augment(text_minority)
                    
                new_text_minority.append(augmented_text_minority)
                new_y_0_minority.append(label_aug_minority) # Assign the label of the original text to the augmented text
                new_y_1_minority.append(1-label_aug_minority) # Assign the label of the original text to the augmented text
                
            new_text_majority = []
            new_y_0_majority = []
            new_y_1_majority = []
            
            for jj in samples_majority: # Go through each sample to be augmented  
                text_majority = x_aug_0_majority[jj]
                label_aug_majority = y_aug_0_majority[jj]
                augmented_text_majority = aug.augment(text_majority)
                    
                new_text_majority.append(augmented_text_majority)
                new_y_0_majority.append(label_aug_majority) # Assign the label of the original text to the augmented text
                new_y_1_majority.append(1-label_aug_majority) # Assign the label of the original text to the augmented text

            new_text = np.append(new_text_minority, new_text_majority)
            new_y_0 = np.append(new_y_0_minority, new_y_0_majority)
            new_y_1 = np.append(new_y_1_minority, new_y_1_majority)
            
            # Append the augmented texts and their corresponding labels to the original data
            x_aug = np.append(x_train, new_text)
            y_0 = np.append(y_train[:,0], new_y_0)
            y_1 = np.append(y_train[:,1], new_y_1)
            y_aug = (np.vstack((y_0,y_1))).T
    
            # Shuffle the order of the augmented dataset
            augmented = pd.DataFrame((np.vstack((x_aug, y_aug[:,0], y_aug[:,1]))).T)
            augmented = augmented.sample(frac=1).reset_index(drop=True)
            
            DA_time = time.time() - DA_start
            augmenter = "aug_"+str(k)
            with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{augmenter},DA Running Time:,{DA_time}\n')
    
            # Convert the dataframe to numpy arrays
            x_train_1 = np.array(augmented.iloc[:,0])
            y_train_1 = np.array(augmented.iloc[:,[1,2]])
            y_train_1 = y_train_1.astype('float32')
            
            
            # Run CNN model
            model_time_start = time.time()
            
            # Preprocess the text
            vfunc = np.vectorize(preprocessing)
            x_train_1 = vfunc(x_train_1)
            x_test_1 = vfunc(x_test)

            # Tokenize the text   
            tokenizer = Tokenizer(num_words=5000) # get the frequency of all tokens and use the 5000 most common ones
            tokenizer.fit_on_texts(x_train_1)
            x_train_1 = tokenizer.texts_to_sequences(x_train_1)
            x_test_1 = tokenizer.texts_to_sequences(x_test_1)
            vocab_size = len(tokenizer.word_index) + 1 # plus the reserved index 0
            word_index = tokenizer.word_index

            # Pad the sequences with 0's
            x_train_1 = pad_sequences(x_train_1, padding='post', maxlen=max_length) 
            x_test_1 = pad_sequences(x_test_1, padding='post', maxlen=max_length)

            # Fit the Transformer model
            mymodel = transformer_model(vocab_size, embedding_dim,max_length,dropout_rate)
            mymodel.fit(x_train_1, y_train_1, epochs=epochs, batch_size=batch_size)

            # Collect and log evaluation metrics
            auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(mymodel, x_test_1, y_test[:,1])
            model_time = time.time() - model_time_start

            with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{augmenter},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{model_time},{len(word_index)},{len(x_aug)}\n')
            
            del tokenizer, mymodel, vocab_size, word_index, x_aug, y_aug, augmented, x_train_1, x_test_1, y_train_1
            gc.collect()
        
        
        time_e = time.time() - time_s
        with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
            fd.write(f'{dis_type},1 iteration 18 DA,{time_e}\n')

    running_time = time.time() - time_start
    with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
            fd.write(f'{dis_type},10 iteration training time,{running_time}\n')

Epoch 1/20
43/43 [==============================] - 77s 2s/step - loss: 0.6189 - auc: 0.7082
Epoch 2/20
43/43 [==============================] - 75s 2s/step - loss: 0.2765 - auc: 0.9535
Epoch 3/20
43/43 [==============================] - 76s 2s/step - loss: 0.0901 - auc: 0.9945
Epoch 4/20
43/43 [==============================] - 76s 2s/step - loss: 0.0263 - auc: 0.9992
Epoch 5/20
43/43 [==============================] - 76s 2s/step - loss: 0.0198 - auc: 0.9998
Epoch 6/20
43/43 [==============================] - 76s 2s/step - loss: 0.0129 - auc: 0.9999
Epoch 7/20
43/43 [==============================] - 75s 2s/step - loss: 0.0165 - auc: 0.9998
Epoch 8/20
43/43 [==============================] - 76s 2s/step - loss: 0.0319 - auc: 0.9994
Epoch 9/20
43/43 [==============================] - 76s 2s/step - loss: 0.0033 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 76s 2s/step - loss: 4.4632e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 75s 2s/step 

43/43 [==============================] - 80s 2s/step - loss: 0.0117 - auc: 0.9999
Epoch 7/20
43/43 [==============================] - 80s 2s/step - loss: 0.0289 - auc: 0.9995
Epoch 8/20
43/43 [==============================] - 80s 2s/step - loss: 0.0058 - auc: 1.0000
Epoch 9/20
43/43 [==============================] - 80s 2s/step - loss: 7.6841e-04 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 80s 2s/step - loss: 3.3771e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 80s 2s/step - loss: 2.1163e-04 - auc: 1.0000
Epoch 12/20
43/43 [==============================] - 80s 2s/step - loss: 2.0824e-04 - auc: 1.0000
Epoch 13/20
43/43 [==============================] - 80s 2s/step - loss: 1.7710e-04 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 80s 2s/step - loss: 1.2961e-04 - auc: 1.0000
Epoch 15/20
43/43 [==============================] - 80s 2s/step - loss: 1.1822e-04 - auc: 1.0000
Epoch 16/20
43/43 [============================

43/43 [==============================] - 82s 2s/step - loss: 4.2095e-04 - auc: 1.0000
Epoch 12/20
43/43 [==============================] - 82s 2s/step - loss: 3.3012e-04 - auc: 1.0000
Epoch 13/20
43/43 [==============================] - 82s 2s/step - loss: 2.3976e-04 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 82s 2s/step - loss: 1.9637e-04 - auc: 1.0000
Epoch 15/20
43/43 [==============================] - 82s 2s/step - loss: 1.8454e-04 - auc: 1.0000
Epoch 16/20
43/43 [==============================] - 82s 2s/step - loss: 1.6564e-04 - auc: 1.0000
Epoch 17/20
43/43 [==============================] - 82s 2s/step - loss: 1.5891e-04 - auc: 1.0000
Epoch 18/20
43/43 [==============================] - 82s 2s/step - loss: 1.3287e-04 - auc: 1.0000
Epoch 19/20
43/43 [==============================] - 82s 2s/step - loss: 1.2818e-04 - auc: 1.0000
Epoch 20/20
43/43 [==============================] - 80s 2s/step - loss: 1.0709e-04 - auc: 1.0000
Epoch 1/20
43/43 [==============

43/43 [==============================] - 78s 2s/step - loss: 1.3828e-04 - auc: 1.0000
Epoch 17/20
43/43 [==============================] - 78s 2s/step - loss: 1.2272e-04 - auc: 1.0000
Epoch 18/20
43/43 [==============================] - 78s 2s/step - loss: 1.0550e-04 - auc: 1.0000
Epoch 19/20
43/43 [==============================] - 78s 2s/step - loss: 9.5165e-05 - auc: 1.0000
Epoch 20/20
43/43 [==============================] - 75s 2s/step - loss: 9.1944e-05 - auc: 1.0000
Epoch 1/20
43/43 [==============================] - 89s 2s/step - loss: 0.6706 - auc: 0.6532
Epoch 2/20
43/43 [==============================] - 88s 2s/step - loss: 0.3902 - auc: 0.9074
Epoch 3/20
43/43 [==============================] - 88s 2s/step - loss: 0.2378 - auc: 0.9656
Epoch 4/20
43/43 [==============================] - 88s 2s/step - loss: 0.0502 - auc: 0.9979
Epoch 5/20
43/43 [==============================] - 88s 2s/step - loss: 0.0205 - auc: 0.9989
Epoch 6/20
43/43 [==============================] - 88s 2

43/43 [==============================] - 86s 2s/step - loss: 0.6560 - auc: 0.6793
Epoch 2/20
43/43 [==============================] - 85s 2s/step - loss: 0.4019 - auc: 0.9018
Epoch 3/20
43/43 [==============================] - 85s 2s/step - loss: 0.1497 - auc: 0.9856
Epoch 4/20
43/43 [==============================] - 86s 2s/step - loss: 0.0516 - auc: 0.9977
Epoch 5/20
43/43 [==============================] - 85s 2s/step - loss: 0.0215 - auc: 0.9998
Epoch 6/20
43/43 [==============================] - 85s 2s/step - loss: 0.0051 - auc: 1.0000
Epoch 7/20
43/43 [==============================] - 86s 2s/step - loss: 0.0019 - auc: 1.0000
Epoch 8/20
43/43 [==============================] - 86s 2s/step - loss: 6.8384e-04 - auc: 1.0000
Epoch 9/20
43/43 [==============================] - 86s 2s/step - loss: 3.9609e-04 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 86s 2s/step - loss: 3.1066e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 86s 2s/step - l

47/47 [==============================] - 90s 2s/step - loss: 0.0156 - auc: 0.9999
Epoch 7/20
47/47 [==============================] - 90s 2s/step - loss: 0.0033 - auc: 1.0000
Epoch 8/20
47/47 [==============================] - 90s 2s/step - loss: 6.1278e-04 - auc: 1.0000
Epoch 9/20
47/47 [==============================] - 90s 2s/step - loss: 3.6796e-04 - auc: 1.0000
Epoch 10/20
47/47 [==============================] - 90s 2s/step - loss: 2.8287e-04 - auc: 1.0000
Epoch 11/20
47/47 [==============================] - 90s 2s/step - loss: 2.2467e-04 - auc: 1.0000
Epoch 12/20
47/47 [==============================] - 90s 2s/step - loss: 1.9358e-04 - auc: 1.0000
Epoch 13/20
47/47 [==============================] - 90s 2s/step - loss: 1.6280e-04 - auc: 1.0000
Epoch 14/20
47/47 [==============================] - 90s 2s/step - loss: 1.4712e-04 - auc: 1.0000
Epoch 15/20
47/47 [==============================] - 90s 2s/step - loss: 1.2741e-04 - auc: 1.0000
Epoch 16/20
47/47 [========================

47/47 [==============================] - 94s 2s/step - loss: 2.0502e-04 - auc: 1.0000
Epoch 12/20
47/47 [==============================] - 94s 2s/step - loss: 1.7673e-04 - auc: 1.0000
Epoch 13/20
47/47 [==============================] - 94s 2s/step - loss: 1.5875e-04 - auc: 1.0000
Epoch 14/20
47/47 [==============================] - 94s 2s/step - loss: 1.3335e-04 - auc: 1.0000
Epoch 15/20
47/47 [==============================] - 94s 2s/step - loss: 1.1906e-04 - auc: 1.0000
Epoch 16/20
47/47 [==============================] - 94s 2s/step - loss: 1.0926e-04 - auc: 1.0000
Epoch 17/20
47/47 [==============================] - 94s 2s/step - loss: 9.7317e-05 - auc: 1.0000
Epoch 18/20
47/47 [==============================] - 94s 2s/step - loss: 9.4647e-05 - auc: 1.0000
Epoch 19/20
47/47 [==============================] - 94s 2s/step - loss: 8.0856e-05 - auc: 1.0000
Epoch 20/20
47/47 [==============================] - 91s 2s/step - loss: 7.5441e-05 - auc: 1.0000
Epoch 1/20
47/47 [==============

47/47 [==============================] - 92s 2s/step - loss: 1.4038e-04 - auc: 1.0000
Epoch 17/20
47/47 [==============================] - 92s 2s/step - loss: 1.2626e-04 - auc: 1.0000
Epoch 18/20
47/47 [==============================] - 92s 2s/step - loss: 1.2417e-04 - auc: 1.0000
Epoch 19/20
47/47 [==============================] - 93s 2s/step - loss: 1.0611e-04 - auc: 1.0000
Epoch 20/20
47/47 [==============================] - 90s 2s/step - loss: 1.0368e-04 - auc: 1.0000
Epoch 1/20
47/47 [==============================] - 70s 1s/step - loss: 0.6440 - auc: 0.6749
Epoch 2/20
47/47 [==============================] - 69s 1s/step - loss: 0.3199 - auc: 0.9372
Epoch 3/20
47/47 [==============================] - 69s 1s/step - loss: 0.1570 - auc: 0.9847
Epoch 4/20
47/47 [==============================] - 69s 1s/step - loss: 0.0500 - auc: 0.9987
Epoch 5/20
47/47 [==============================] - 70s 1s/step - loss: 0.0096 - auc: 1.0000
Epoch 6/20
47/47 [==============================] - 69s 1

47/47 [==============================] - 84s 2s/step - loss: 0.6394 - auc: 0.6842
Epoch 2/20
47/47 [==============================] - 83s 2s/step - loss: 0.3376 - auc: 0.9297
Epoch 3/20
47/47 [==============================] - 83s 2s/step - loss: 0.1763 - auc: 0.9800
Epoch 4/20
47/47 [==============================] - 83s 2s/step - loss: 0.0672 - auc: 0.9978
Epoch 5/20
47/47 [==============================] - 83s 2s/step - loss: 0.0168 - auc: 0.9992
Epoch 6/20
47/47 [==============================] - 84s 2s/step - loss: 0.0088 - auc: 1.0000
Epoch 7/20
47/47 [==============================] - 83s 2s/step - loss: 0.0022 - auc: 1.0000
Epoch 8/20
47/47 [==============================] - 83s 2s/step - loss: 5.5295e-04 - auc: 1.0000
Epoch 9/20
47/47 [==============================] - 83s 2s/step - loss: 2.7090e-04 - auc: 1.0000
Epoch 10/20
47/47 [==============================] - 83s 2s/step - loss: 2.2293e-04 - auc: 1.0000
Epoch 11/20
47/47 [==============================] - 83s 2s/step - l

47/47 [==============================] - 83s 2s/step - loss: 0.0139 - auc: 0.9992
Epoch 7/20
47/47 [==============================] - 84s 2s/step - loss: 0.0021 - auc: 1.0000
Epoch 8/20
47/47 [==============================] - 83s 2s/step - loss: 5.4905e-04 - auc: 1.0000
Epoch 9/20
47/47 [==============================] - 83s 2s/step - loss: 3.7188e-04 - auc: 1.0000
Epoch 10/20
47/47 [==============================] - 83s 2s/step - loss: 2.9181e-04 - auc: 1.0000
Epoch 11/20
47/47 [==============================] - 83s 2s/step - loss: 2.7846e-04 - auc: 1.0000
Epoch 12/20
47/47 [==============================] - 83s 2s/step - loss: 2.0488e-04 - auc: 1.0000
Epoch 13/20
47/47 [==============================] - 83s 2s/step - loss: 1.8745e-04 - auc: 1.0000
Epoch 14/20
47/47 [==============================] - 83s 2s/step - loss: 1.6291e-04 - auc: 1.0000
Epoch 15/20
47/47 [==============================] - 83s 2s/step - loss: 1.4528e-04 - auc: 1.0000
Epoch 16/20
47/47 [========================

## Parallel Computing

In [15]:
# pip install dill

In [16]:
# import dill as pickle

In [17]:
# import nltk
# nltk.download('omw')

In [18]:
# Parallelizing the computation for all 16 disease conditions

# Parallel(n_jobs=16,backend="threading")(delayed(main_cnn)(x) for x in range(16))
# Parallel(n_jobs=-2,backend="threading")(delayed(main_cnn)(x) for x in [13])

In [19]:
time_2 = time.time() - time_1

with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
            fd.write(f'Total running time, {time_2}\n')